# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fa170446d90>
├── 'a' --> tensor([[-0.1059,  0.7505, -0.8255],
│                   [ 1.7596, -0.1591, -1.0545]])
└── 'x' --> <FastTreeValue 0x7fa172509940>
    └── 'c' --> tensor([[ 0.4189,  0.0130, -0.5672, -1.8358],
                        [ 0.0634, -0.5013,  0.0962, -0.0224],
                        [ 2.1407, -0.7129, -0.3623, -1.2651]])

In [4]:
t.a

tensor([[-0.1059,  0.7505, -0.8255],
        [ 1.7596, -0.1591, -1.0545]])

In [5]:
%timeit t.a

59.1 ns ± 0.0161 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fa170446d90>
├── 'a' --> tensor([[ 0.8346,  0.9044, -0.3395],
│                   [-0.7150,  0.3680, -0.7611]])
└── 'x' --> <FastTreeValue 0x7fa172509940>
    └── 'c' --> tensor([[ 0.4189,  0.0130, -0.5672, -1.8358],
                        [ 0.0634, -0.5013,  0.0962, -0.0224],
                        [ 2.1407, -0.7129, -0.3623, -1.2651]])

In [7]:
%timeit t.a = new_value

64 ns ± 0.0242 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.1059,  0.7505, -0.8255],
               [ 1.7596, -0.1591, -1.0545]]),
    x: Batch(
           c: tensor([[ 0.4189,  0.0130, -0.5672, -1.8358],
                      [ 0.0634, -0.5013,  0.0962, -0.0224],
                      [ 2.1407, -0.7129, -0.3623, -1.2651]]),
       ),
)

In [10]:
b.a

tensor([[-0.1059,  0.7505, -0.8255],
        [ 1.7596, -0.1591, -1.0545]])

In [11]:
%timeit b.a

58.1 ns ± 0.0264 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.9929, -0.7218, -1.0869],
               [-0.0770, -0.4405, -0.2078]]),
    x: Batch(
           c: tensor([[ 0.4189,  0.0130, -0.5672, -1.8358],
                      [ 0.0634, -0.5013,  0.0962, -0.0224],
                      [ 2.1407, -0.7129, -0.3623, -1.2651]]),
       ),
)

In [13]:
%timeit b.a = new_value

488 ns ± 1.36 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

885 ns ± 3.83 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.3 µs ± 48.7 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

135 µs ± 197 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

140 µs ± 372 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fa0c3088d90>
├── 'a' --> tensor([[[-0.1059,  0.7505, -0.8255],
│                    [ 1.7596, -0.1591, -1.0545]],
│           
│                   [[-0.1059,  0.7505, -0.8255],
│                    [ 1.7596, -0.1591, -1.0545]],
│           
│                   [[-0.1059,  0.7505, -0.8255],
│                    [ 1.7596, -0.1591, -1.0545]],
│           
│                   [[-0.1059,  0.7505, -0.8255],
│                    [ 1.7596, -0.1591, -1.0545]],
│           
│                   [[-0.1059,  0.7505, -0.8255],
│                    [ 1.7596, -0.1591, -1.0545]],
│           
│                   [[-0.1059,  0.7505, -0.8255],
│                    [ 1.7596, -0.1591, -1.0545]],
│           
│                   [[-0.1059,  0.7505, -0.8255],
│                    [ 1.7596, -0.1591, -1.0545]],
│           
│                   [[-0.1059,  0.7505, -0.8255],
│                    [ 1.7596, -0.1591, -1.0545]]])
└── 'x' --> <FastTreeValue 0x7fa0c3109cd0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.4 µs ± 35.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fa1448a7e80>
├── 'a' --> tensor([[-0.1059,  0.7505, -0.8255],
│                   [ 1.7596, -0.1591, -1.0545],
│                   [-0.1059,  0.7505, -0.8255],
│                   [ 1.7596, -0.1591, -1.0545],
│                   [-0.1059,  0.7505, -0.8255],
│                   [ 1.7596, -0.1591, -1.0545],
│                   [-0.1059,  0.7505, -0.8255],
│                   [ 1.7596, -0.1591, -1.0545],
│                   [-0.1059,  0.7505, -0.8255],
│                   [ 1.7596, -0.1591, -1.0545],
│                   [-0.1059,  0.7505, -0.8255],
│                   [ 1.7596, -0.1591, -1.0545],
│                   [-0.1059,  0.7505, -0.8255],
│                   [ 1.7596, -0.1591, -1.0545],
│                   [-0.1059,  0.7505, -0.8255],
│                   [ 1.7596, -0.1591, -1.0545]])
└── 'x' --> <FastTreeValue 0x7fa0c30b9f40>
    └── 'c' --> tensor([[ 0.4189,  0.0130, -0.5672, -1.8358],
                        [ 0.0634, -0.5013,  0.0962, -0.0224],
                 

In [23]:
%timeit t_cat(trees)

30.7 µs ± 124 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

60 µs ± 39.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.1059,  0.7505, -0.8255],
                [ 1.7596, -0.1591, -1.0545]],
       
               [[-0.1059,  0.7505, -0.8255],
                [ 1.7596, -0.1591, -1.0545]],
       
               [[-0.1059,  0.7505, -0.8255],
                [ 1.7596, -0.1591, -1.0545]],
       
               [[-0.1059,  0.7505, -0.8255],
                [ 1.7596, -0.1591, -1.0545]],
       
               [[-0.1059,  0.7505, -0.8255],
                [ 1.7596, -0.1591, -1.0545]],
       
               [[-0.1059,  0.7505, -0.8255],
                [ 1.7596, -0.1591, -1.0545]],
       
               [[-0.1059,  0.7505, -0.8255],
                [ 1.7596, -0.1591, -1.0545]],
       
               [[-0.1059,  0.7505, -0.8255],
                [ 1.7596, -0.1591, -1.0545]]]),
    x: Batch(
           c: tensor([[[ 0.4189,  0.0130, -0.5672, -1.8358],
                       [ 0.0634, -0.5013,  0.0962, -0.0224],
                       [ 2.1407, -0.7129, -0.3623, -1.2651]],
         

In [26]:
%timeit Batch.stack(batches)

79.1 µs ± 180 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.1059,  0.7505, -0.8255],
               [ 1.7596, -0.1591, -1.0545],
               [-0.1059,  0.7505, -0.8255],
               [ 1.7596, -0.1591, -1.0545],
               [-0.1059,  0.7505, -0.8255],
               [ 1.7596, -0.1591, -1.0545],
               [-0.1059,  0.7505, -0.8255],
               [ 1.7596, -0.1591, -1.0545],
               [-0.1059,  0.7505, -0.8255],
               [ 1.7596, -0.1591, -1.0545],
               [-0.1059,  0.7505, -0.8255],
               [ 1.7596, -0.1591, -1.0545],
               [-0.1059,  0.7505, -0.8255],
               [ 1.7596, -0.1591, -1.0545],
               [-0.1059,  0.7505, -0.8255],
               [ 1.7596, -0.1591, -1.0545]]),
    x: Batch(
           c: tensor([[ 0.4189,  0.0130, -0.5672, -1.8358],
                      [ 0.0634, -0.5013,  0.0962, -0.0224],
                      [ 2.1407, -0.7129, -0.3623, -1.2651],
                      [ 0.4189,  0.0130, -0.5672, -1.8358],
                      [ 0.0634, -

In [28]:
%timeit Batch.cat(batches)

144 µs ± 546 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

312 µs ± 1.07 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
